# **B2B Courier Charges Accuracy Analysis**

In [59]:
#importing necessary libraryf
import pandas as pd

In [60]:
#Loading related datasets for analysis
order_report = pd.read_csv('Order Report.csv')
sku_master = pd.read_csv('SKU Master.csv')
pincode_mapping = pd.read_csv('pincodes.csv')
courier_invoice = pd.read_csv('Invoice.csv')
courier_company_rates = pd.read_csv('Courier Company - Rates.csv')

In [61]:
# Order-report dataset overview
print("Order Report :")
order_report.head()

Order Report :


,ExternOrderNo,SKU,Order Qty,Unnamed: 3,Unnamed: 4
0,2001827036,8904223818706,1.0,NaN,NaN
1,2001827036,8904223819093,1.0,NaN,NaN
2,2001827036,8904223819109,1.0,NaN,NaN
3,2001827036,8904223818430,1.0,NaN,NaN
4,2001827036,8904223819277,1.0,NaN,NaN


Here SKU - Stock Keeping Unit ID

In [62]:
# SKU Master dataset overview
print("\nSKU Master:")
print(sku_master.head())


SKU Master:
             SKU  Weight (g)  Unnamed: 2  Unnamed: 3  Unnamed: 4
0  8904223815682         210         NaN         NaN         NaN
1  8904223815859         165         NaN         NaN         NaN
2  8904223815866         113         NaN         NaN         NaN
3  8904223815873          65         NaN         NaN         NaN
4  8904223816214         120         NaN         NaN         NaN


In [63]:
# Pincode Mapping dataset overview
print("\nPincode Mapping:")
print(pincode_mapping.head())


Pincode Mapping:
   Warehouse Pincode  Customer Pincode Zone  Unnamed: 3  Unnamed: 4
0             121003            507101    d         NaN         NaN
1             121003            486886    d         NaN         NaN
2             121003            532484    d         NaN         NaN
3             121003            143001    b         NaN         NaN
4             121003            515591    d         NaN         NaN


In [64]:
# Courier Invoice dataset overview
print("\nCourier Invoice:")
print(courier_invoice.head())


Courier Invoice:
        AWB Code    Order ID  Charged Weight  Warehouse Pincode  \
0  1091117222124  2001806232            1.30             121003   
1  1091117222194  2001806273            1.00             121003   
2  1091117222931  2001806408            2.50             121003   
3  1091117223244  2001806458            1.00             121003   
4  1091117229345  2001807012            0.15             121003   

   Customer Pincode Zone Type of Shipment  Billing Amount (Rs.)  
0            507101    d  Forward charges                 135.0  
1            486886    d  Forward charges                  90.2  
2            532484    d  Forward charges                 224.6  
3            143001    b  Forward charges                  61.3  
4            515591    d  Forward charges                  45.4  


In [65]:
# Check for missing values
print("\nMissing values in Website Order Report:")
print(order_report.isnull().sum())
print("\nMissing values in SKU Master:")
print(sku_master.isnull().sum())
print("\nMissing values in Pincode Mapping:")
print(pincode_mapping.isnull().sum())
print("\nMissing values in Courier Invoice:")
print(courier_invoice.isnull().sum())
print("\nMissing values in courier company rates:")
print(courier_company_rates.isnull().sum())


Missing values in Website Order Report:
ExternOrderNo      0
SKU                0
Order Qty          0
Unnamed: 3       400
Unnamed: 4       400
dtype: int64

Missing values in SKU Master:
SKU            0
Weight (g)     0
Unnamed: 2    66
Unnamed: 3    66
Unnamed: 4    66
dtype: int64

Missing values in Pincode Mapping:
Warehouse Pincode      0
Customer Pincode       0
Zone                   0
Unnamed: 3           124
Unnamed: 4           124
dtype: int64

Missing values in Courier Invoice:
AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64

Missing values in courier company rates:
fwd_a_fixed         0
fwd_a_additional    0
fwd_b_fixed         0
fwd_b_additional    0
fwd_c_fixed         0
fwd_c_additional    0
fwd_d_fixed         0
fwd_d_additional    0
fwd_e_fixed         0
fwd_e_additional    0
rto_a_fixed        

Here, we can notice that All the unamed columns have Null values

so we will clean the dataset by Dropping the unnamed columns

In [66]:
# Remove unnamed columns from the Website Order Report DataFrame
order_report = order_report.drop(columns=['Unnamed: 3', 'Unnamed: 4'])

In [67]:
# Remove unnamed columns from the SKU Master DataFrame
sku_master = sku_master.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])

In [68]:
# Remove unnamed columns from the Pincode Mapping DataFrame
pincode_mapping = pincode_mapping.drop(columns=['Unnamed: 3', 'Unnamed: 4'])

Merging the order report and SKU master datasets according to the common SKU column:

In [69]:
merged_data=pd.merge(order_report,sku_master,on='SKU')
print(merged_data.head())

   ExternOrderNo            SKU  Order Qty  Weight (g)
0     2001827036  8904223818706        1.0         127
1     2001827036  8904223819093        1.0         150
2     2001827036  8904223819109        1.0         100
3     2001827036  8904223818430        1.0         165
4     2001827036  8904223819277        1.0         350


Now let's merge the Pincode_mapping Dataset and Courier Invoice Dataset

before that we will remove the remove the duplicates of the Pincode mapping dataset using Customer_pincode subset column as Subset

In [70]:
abc_courier = pincode_mapping.drop_duplicates(subset=['Customer Pincode'])
courier_abc= courier_invoice[['Order ID', 'Customer Pincode','Type of Shipment']]
pincodes= courier_abc.merge(abc_courier,on='Customer Pincode')
print(pincodes.head())

     Order ID  Customer Pincode Type of Shipment  Warehouse Pincode Zone
0  2001806232            507101  Forward charges             121003    d
1  2001806273            486886  Forward charges             121003    d
2  2001806408            532484  Forward charges             121003    d
3  2001806458            143001  Forward charges             121003    b
4  2001807012            515591  Forward charges             121003    d


1.We first extract the unique customer pin codes from the pincode mapping dataset and create a new DataFrame called “abc_courier” to store this information.

2.We then select specific columns (“Order ID”, “Customer Pincode”, “Type of Shipment”) from the courier_invoice dataset and create a new DataFrame called “courier_abc” to store this subset of data.

3.We then merge the ‘courier_abc’ DataFrame with the ‘abc_courier’ DataFrame based on the ‘Customer Pincode’ column. This merge operation helps us associate customer pin codes with their respective orders and shipping types. The resulting DataFrame is named ‘pincodes’

In [71]:
# Assuming the column name in merged_data is different, like 'ExternOrderNo'
merged2 = merged_data.merge(pincodes, left_on='ExternOrderNo', right_on='Order ID')

In [72]:
print(merged2)

     ExternOrderNo            SKU  Order Qty  Weight (g)    Order ID  \
0       2001827036  8904223818706        1.0         127  2001827036   
1       2001827036  8904223819093        1.0         150  2001827036   
2       2001827036  8904223819109        1.0         100  2001827036   
3       2001827036  8904223818430        1.0         165  2001827036   
4       2001827036  8904223819277        1.0         350  2001827036   
..             ...            ...        ...         ...         ...   
396     2001806229  8904223818942        1.0         133  2001806229   
397     2001806229  8904223818850        1.0         240  2001806229   
398     2001806226  8904223818850        2.0         240  2001806226   
399     2001806210  8904223816214        1.0         120  2001806210   
400     2001806210  8904223818874        1.0         100  2001806210   

     Customer Pincode Type of Shipment  Warehouse Pincode Zone  
0              173213  Forward charges             121003    e  
1    

In [73]:
merged2['Weights (Kgs)'] = merged2['Weight (g)'] / 1000

In [74]:
print(merged2.head())

   ExternOrderNo            SKU  Order Qty  Weight (g)    Order ID  \
0     2001827036  8904223818706        1.0         127  2001827036   
1     2001827036  8904223819093        1.0         150  2001827036   
2     2001827036  8904223819109        1.0         100  2001827036   
3     2001827036  8904223818430        1.0         165  2001827036   
4     2001827036  8904223819277        1.0         350  2001827036   

   Customer Pincode Type of Shipment  Warehouse Pincode Zone  Weights (Kgs)  
0            173213  Forward charges             121003    e          0.127  
1            173213  Forward charges             121003    e          0.150  
2            173213  Forward charges             121003    e          0.100  
3            173213  Forward charges             121003    e          0.165  
4            173213  Forward charges             121003    e          0.350  


In [75]:
def weight_slab(weight):
    i = round(weight % 1, 1)
    if i == 0.0:
        return weight
    elif i > 0.5:
        return int(weight) + 1.0
    else:
        return int(weight) + 0.5

merged2['Weight Slab (KG)'] = merged2['Weights (Kgs)'].apply(weight_slab)
courier_invoice['Weight Slab Charged by Courier Company']=(courier_invoice['Charged Weight']).apply(weight_slab)

The weight_slab() function is defined to determine the weight slab based on the weight of the shipment. It takes the input weight and applies certain conditions to calculate the weight slab. Below is how it works:

1. xThe function first calculates the remainder of the weight divided by 1 and rounds it to one decimal place. If the remainder is 0.0, it means the weight is a multiple of 1 KG, and the function returns the weight as it is.

2. If the remainder is greater than 0.5, it means that the weight exceeds the next half KG slab. In this case, the function rounds the weight to the nearest integer and adds 1.0 to it, which represents the next heavier slab.

3. If the remainder is less than or equal to 0.5, it means the weight falls into the current half-KG bracket. In this case, the function rounds the weight to the nearest integer and adds 0.5 to it, which represents the current weight slab.

In [77]:
courier_invoice = courier_invoice.rename(columns={'Zone': 'Delivery Zone Charged by Courier Company'})
merged2 = merged2.rename(columns={'Zone': 'Delivery Zone As Per ABC'})
merged2 = merged2.rename(columns={'Weight Slab (KG)': 'Weight Slab As Per ABC'})

In [78]:
merged2.head()

,ExternOrderNo,SKU,Order Qty,Weight (g),Order ID,Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone As Per ABC,Weights (Kgs),Weight Slab As Per ABC
0,2001827036,8904223818706,1.0,127,2001827036,173213,Forward charges,121003,e,0.127,0.5
1,2001827036,8904223819093,1.0,150,2001827036,173213,Forward charges,121003,e,0.150,0.5
2,2001827036,8904223819109,1.0,100,2001827036,173213,Forward charges,121003,e,0.100,0.5
3,2001827036,8904223818430,1.0,165,2001827036,173213,Forward charges,121003,e,0.165,0.5
4,2001827036,8904223819277,1.0,350,2001827036,173213,Forward charges,121003,e,0.350,0.5


In [79]:
total_expected_charge = []


An empty list called total_expected_charge is created to store the calculated expected charges for each order.

In [81]:
for _, row in merged2.iterrows():
    fwd_category = 'fwd_' + row['Delivery Zone As Per ABC']
    fwd_fixed = courier_company_rates.at[0, fwd_category + '_fixed']
    fwd_additional = courier_company_rates.at[0, fwd_category + '_additional']
    rto_category = 'rto_' + row['Delivery Zone As Per ABC']
    rto_fixed = courier_company_rates.at[0, rto_category + '_fixed']
    rto_additional = courier_company_rates.at[0, rto_category + '_additional']

    weight_slab = row['Weight Slab As Per ABC']

    if row['Type of Shipment'] == 'Forward charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * fwd_additional)
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * (fwd_additional + rto_additional))
    else:
        total_expected_charge.append(0)


This line starts a loop that iterates through each row of the merged2 DataFrame.

iterrows() is a pandas function that returns an iterator yielding both the index and the row data for each row in the DataFrame.

The _ is used as a placeholder for the index, as it's not needed in this case

Upcoming lines determine the courier charges based on the delivery zone.

* fwd_category and rto_category are created by concatenating 'fwd_' or 'rto_' with the 'Delivery Zone As Per ABC' from the current row. These represent the categories for forward and return charges.

* fwd_fixed, fwd_additional, rto_fixed, and rto_additional are fetched from the courier_company_rates DataFrame using .at, which accesses a single value by row and column labels.

* These variables store the fixed and additional charges for forward and return shipments for the specific delivery zone.

    weight_slab = row['Weight Slab As Per ABC']

    if row['Type of Shipment'] == 'Forward charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * fwd_additional)
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * (fwd_additional + rto_additional))
    else:
        total_expected_charge.append(0)
This block calculates the expected charge based on the 'Type of Shipment'.

- If the 'Type of Shipment' is 'Forward charges', it calculates the additional weight and adds the fixed and additional charges for forward shipment to the total_expected_charge list.

- If the 'Type of Shipment' is 'Forward and RTO charges', it calculates the additional weight and adds the fixed and additional charges for both forward and return shipments to the total_expected_charge list.

- If the 'Type of Shipment' is neither of the above, it adds 0 to the total_expected_charge list.

In [82]:
merged2['Expected Charge as per ABC'] = total_expected_charge
print(merged2.head())

   ExternOrderNo            SKU  Order Qty  Weight (g)    Order ID  \
0     2001827036  8904223818706        1.0         127  2001827036   
1     2001827036  8904223819093        1.0         150  2001827036   
2     2001827036  8904223819109        1.0         100  2001827036   
3     2001827036  8904223818430        1.0         165  2001827036   
4     2001827036  8904223819277        1.0         350  2001827036   

   Customer Pincode Type of Shipment  Warehouse Pincode  \
0            173213  Forward charges             121003   
1            173213  Forward charges             121003   
2            173213  Forward charges             121003   
3            173213  Forward charges             121003   
4            173213  Forward charges             121003   

  Delivery Zone As Per ABC  Weights (Kgs)  Weight Slab As Per ABC  \
0                        e          0.127                     0.5   
1                        e          0.150                     0.5   
2                

- This line adds the `total_expected_charge` list as a new column named 'Expected Charge as per ABC' to the `merged2` DataFrame.
- Finally, `print(merged2.head())` displays the first few rows of the updated `merged2` DataFrame, showing the calculated expected charges.


Now let’s merge it with the courier invoice to display the final dataframe:

In [83]:
merged_output = merged2.merge(courier_invoice, on='Order ID')
print(merged_output.head())

   ExternOrderNo            SKU  Order Qty  Weight (g)    Order ID  \
0     2001827036  8904223818706        1.0         127  2001827036   
1     2001827036  8904223819093        1.0         150  2001827036   
2     2001827036  8904223819109        1.0         100  2001827036   
3     2001827036  8904223818430        1.0         165  2001827036   
4     2001827036  8904223819277        1.0         350  2001827036   

   Customer Pincode_x Type of Shipment_x  Warehouse Pincode_x  \
0              173213    Forward charges               121003   
1              173213    Forward charges               121003   
2              173213    Forward charges               121003   
3              173213    Forward charges               121003   
4              173213    Forward charges               121003   

  Delivery Zone As Per ABC  Weights (Kgs)  Weight Slab As Per ABC  \
0                        e          0.127                     0.5   
1                        e          0.150         

Now let’s calculate the differences in charges and expected charges for each order:

In [84]:
df_diff = merged_output
df_diff['Difference (Rs.)'] = df_diff['Billing Amount (Rs.)'] - df_diff['Expected Charge as per ABC']

df_new = df_diff[['Order ID', 'Difference (Rs.)', 'Expected Charge as per ABC']]

print(df_new.head())

     Order ID  Difference (Rs.)  Expected Charge as per ABC
0  2001827036              61.3                        56.6
1  2001827036              61.3                        56.6
2  2001827036              61.3                        56.6
3  2001827036              61.3                        56.6
4  2001827036              61.3                        56.6


In [87]:
# Calculate the total orders in each category
total_correctly_charged = len(df_new[df_new['Difference (Rs.)'] == 0])
total_overcharged = len(df_new[df_new['Difference (Rs.)'] > 0])
total_undercharged = len(df_new[df_new['Difference (Rs.)'] < 0])

In [88]:
# Calculate the total amount in each category
amount_overcharged = abs(df_new[df_new['Difference (Rs.)'] > 0]['Difference (Rs.)'].sum())
amount_undercharged = df_new[df_new['Difference (Rs.)'] < 0]['Difference (Rs.)'].sum()
amount_correctly_charged = df_new[df_new['Difference (Rs.)'] == 0]['Expected Charge as per ABC'].sum()

In [89]:
# Create a new DataFrame for the summary
summary_data = {'Description': ['Total Orders where ABC has been correctly charged',
                                'Total Orders where ABC has been overcharged',
                                'Total Orders where ABC has been undercharged'],
                'Count': [total_correctly_charged, total_overcharged, total_undercharged],
                'Amount (Rs.)': [amount_correctly_charged, amount_overcharged, amount_undercharged]}

df_summary = pd.DataFrame(summary_data)

print(df_summary)

                                         Description  Count  Amount (Rs.)
0  Total Orders where ABC has been correctly charged     12         507.6
1        Total Orders where ABC has been overcharged    382       33750.5
2       Total Orders where ABC has been undercharged      7        -165.2


We can also visualize the proportion of errors as shown below:

In [90]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Pie(labels=df_summary['Description'],
                            values=df_summary['Count'],
                            textinfo='label+percent',
                            hole=0.4))
fig.update_layout(title='Proportion')

fig.show()

This diagram is a donut chart that represents the proportion of total orders based on their charging accuracy for a category labeled "ABC."

Key Observations:

1. Overcharged Orders (Blue - 95.3%)

The vast majority of the orders (95.3%) have been overcharged.

2. Correctly Charged Orders (Red - 2.99%)

A small portion of the orders (2.99%) were charged correctly.

3. Undercharged Orders (Green - 1.75%)

An even smaller portion (1.75%) were undercharged.

Insights:

The overcharging rate is significantly high, meaning that ABC is frequently being billed more than it should be.

The percentage of correctly charged orders is quite low.

Undercharging is minimal compared to overcharging.

This analysis suggests a potential issue with pricing or billing, and further investigation might be needed to correct the overcharging problem.